In [3]:
import pandas as pd
import math
import numpy as np

%matplotlib inline

from pathlib import Path
from omegaconf import OmegaConf
import scipy.stats as st

In [8]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "valid_log.csv").is_file():
        cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
        cfg = OmegaConf.to_container(cfg)
        num_epochs = cfg['optim']['epochs']

        valid_log = pd.read_csv(run / "valid_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-3) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("valid_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    len(list(Path('.').glob('*')))
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def check_image_size(run, image_size=480):
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)
    conf_image_size = cfg['data']['image_size']
    if int(conf_image_size) != image_size:
        print("Different image size in conf. Image size in conf: {}, Image size: {}".format(conf_image_size, image_size))

def collect_one(model_name, run, csv_file, image_size=480):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    check_image_size(run, image_size=image_size)
    
    cfg = OmegaConf.load(run / '.hydra' / 'config.yaml')
    cfg = OmegaConf.to_container(cfg)

    run_number = cfg['data']['train']['cross_val_bucket_validation_index']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / 'test_predictions' / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = run_number
    data['inv_temp'] = inv_temp
    data['regime'] = regime
    
    return data

def collect_all(model_name, root, csv_file, regimes=['0.01', '0.02', '0.05', '0.1', '0.2', '0.25', '1.0'], image_size=480):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    metrics.append(collect_one(model_name, run, csv_file, image_size=image_size))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

def collect_all_regimes(model_name, root, csv_file, regimes=['0.1', '0.2', '0.5', '1.0'], image_size=480):
    root = Path(root)

    metrics = []
    for run in list(root.glob("run-*")):
        if root.name.rsplit("-", 1)[1] in regimes:
            metrics.append(collect_one(model_name, run, csv_file, image_size=image_size))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [5]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], model_group[1], model_group[2], model_group[3]
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], t_student=False):
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate(['mean', 'std'])
    
    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [5]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t')),
}

In [6]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [7]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,1.0,1.0,52.31,36.16,41.72,8.55
1,H-UNet-BASE-SWTA,0,5.0,1.0,49.74,33.87,42.40,8.16
2,H-UNet-BASE-SWTA,0,10.0,1.0,45.79,30.24,45.59,8.88
3,H-UNet-BASE-SWTA,0,20.0,1.0,49.81,33.74,38.32,7.20
4,H-UNet-BASE-SWTA,0,50.0,1.0,49.98,34.01,42.11,7.88
5,H-UNet-BASE-SWTA,0,100.0,1.0,49.48,33.46,40.05,7.27
6,H-UNet-BASE-SWTA-T,0,1.0,1.0,45.60,30.22,44.54,9.98
7,H-UNet-BASE-SWTA-T,0,5.0,1.0,48.07,32.26,54.12,10.52
8,H-UNet-BASE-SWTA-T,0,10.0,1.0,49.61,33.60,42.02,8.29
9,H-UNet-BASE-SWTA-T,0,20.0,1.0,50.11,34.03,42.04,7.84


Dice     Jaccard     Hausdorff Distance  \
                                     mean std    mean std               mean   
Model              Inv Temp Regime                                             
H-UNet-BASE-SWTA   1.0      1.0     52.31 NaN   36.16 NaN              41.72   
                   5.0      1.0     49.74 NaN   33.87 NaN              42.40   
                   10.0     1.0     45.79 NaN   30.24 NaN              45.59   
                   20.0     1.0     49.81 NaN   33.74 NaN              38.32   
                   50.0     1.0     49.98 NaN   34.01 NaN              42.11   
                   100.0    1.0     49.48 NaN   33.46 NaN              40.05   
H-UNet-BASE-SWTA-T 1.0      1.0     45.60 NaN   30.22 NaN              44.54   
                   5.0      1.0     48.07 NaN   32.26 NaN              54.12   
                   10.0     1.0     49.61 NaN   33.60 NaN              42.02   
                   20.0     1.0     50.11 NaN   34.03 NaN              42.04   
                   50.0     1.0     47.67 NaN   31.89 NaN              43.38   
                   100.0    1.0     49.24 NaN   33.43 NaN              41.65   

                                       Average Surface Distance      
                                   std                     mean std  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   1.0      1.0    NaN                     8.55 NaN  
                   5.0      1.0    NaN                     8.16 NaN  
                   10.0     1.0    NaN                     8.88 NaN  
                   20.0     1.0    NaN                     7.20 NaN  
                   50.0     1.0    NaN                     7.88 NaN  
                   100.0    1.0    NaN                     7.27 NaN  
H-UNet-BASE-SWTA-T 1.0      1.0    NaN                     9.98 NaN  
                   5.0      1.0    NaN                    10.52 NaN  
                   10.0     1.0    NaN                     8.29 NaN  
                   20.0     1.0    NaN                     7.84 NaN  
                   50.0     1.0    NaN                     8.20 NaN  
                   100.0    1.0    NaN                     8.29 NaN

In [8]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    #'H-UNet-BASE-SWTA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft_fromBestDice')),
    #'H-UNet-BASE-SWTA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft_fromBestDice')),
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/').glob('hunet-swta_t_ft')),
}

In [9]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [10]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT,0,1.0,0.2,78.29,65.23,25.62,3.60
1,H-UNet-BASE-SWTA-FT,0,5.0,0.2,79.96,67.77,27.76,3.96
2,H-UNet-BASE-SWTA-FT,0,10.0,0.2,81.71,70.05,23.32,3.45
3,H-UNet-BASE-SWTA-FT,0,20.0,0.2,81.64,69.71,19.47,2.40
4,H-UNet-BASE-SWTA-FT,0,50.0,0.2,81.50,69.69,22.19,3.04
...,...,...,...,...,...,...,...,...
115,H-UNet-BASE-SWTA-T-FT,9,5.0,0.2,80.07,68.01,22.94,3.58
116,H-UNet-BASE-SWTA-T-FT,9,10.0,0.2,79.62,67.11,26.51,3.54
117,H-UNet-BASE-SWTA-T-FT,9,20.0,0.2,80.61,68.63,28.82,3.85
118,H-UNet-BASE-SWTA-T-FT,9,50.0,0.2,79.22,66.88,27.03,3.87


Dice           Jaccard            \
                                         mean       std    mean       std   
Model                 Inv Temp Regime                                       
H-UNet-BASE-SWTA-FT   1.0      0.2     77.890  1.997649  64.949  2.580833   
                      5.0      0.2     79.583  1.966842  67.351  2.601664   
                      10.0     0.2     79.743  1.593606  67.467  2.118732   
                      20.0     0.2     80.649  1.280663  68.642  1.708799   
                      50.0     0.2     80.322  1.563833  68.282  2.008149   
                      100.0    0.2     80.417  2.325621  68.408  3.034636   
H-UNet-BASE-SWTA-T-FT 1.0      0.2     78.210  1.914233  65.403  2.549466   
                      5.0      0.2     80.459  1.786520  68.528  2.441210   
                      10.0     0.2     81.042  1.461079  69.245  1.992058   
                      20.0     0.2     81.511  2.092566  69.898  2.830684   
                      50.0     0.2     81.150  1.887467  69.536  2.490543   
                      100.0    0.2     80.874  2.195269  69.032  2.904953   

                                      Hausdorff Distance            \
                                                    mean       std   
Model                 Inv Temp Regime                                
H-UNet-BASE-SWTA-FT   1.0      0.2                28.680  3.489909   
                      5.0      0.2                26.779  5.198413   
                      10.0     0.2                26.086  5.496421   
                      20.0     0.2                24.566  4.580646   
                      50.0     0.2                25.608  2.128227   
                      100.0    0.2                25.240  4.685384   
H-UNet-BASE-SWTA-T-FT 1.0      0.2                28.162  5.176704   
                      5.0      0.2                25.713  3.538424   
                      10.0     0.2                24.659  4.680369   
                      20.0     0.2                25.000  4.665269   
                      50.0     0.2                24.798  3.130129   
                      100.0    0.2                25.238  5.418556   

                                      Average Surface Distance            
                                                          mean       std  
Model                 Inv Temp Regime                                     
H-UNet-BASE-SWTA-FT   1.0      0.2                       4.102  0.595871  
                      5.0      0.2                       3.807  0.716505  
                      10.0     0.2                       3.717  0.720140  
                      20.0     0.2                       3.346  0.760675  
                      50.0     0.2                       3.521  0.304939  
                      100.0    0.2                       3.545  0.652521  
H-UNet-BASE-SWTA-T-FT 1.0      0.2                       4.079  0.975676  
                      5.0      0.2                       3.632  0.391828  
                      10.0     0.2                       3.455  0.584964  
                      20.0     0.2                       3.445  0.671669  
                      50.0     0.2                       3.545  0.526566  
                      100.0    0.2                       3.458  0.742949

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [11]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

INV_TEMP_GlaS=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_GlaS)).glob('run-*')),
}

In [12]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [13]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_1874990/265248669.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA,0,20.0,1.0,49.81,33.74,38.32,7.20
1,H-UNet-BASE-SWTA-T,0,20.0,1.0,50.11,34.03,42.04,7.84
2,H-UNet-HPCA,0,1.0,1.0,33.72,20.47,NaN,NaN
3,H-UNet-HPCA-T,0,1.0,1.0,34.84,21.30,NaN,NaN


Dice     Jaccard     Hausdorff Distance  \
                                     mean std    mean std               mean   
Model              Inv Temp Regime                                             
H-UNet-BASE-SWTA   20.0     1.0     49.81 NaN   33.74 NaN              38.32   
H-UNet-BASE-SWTA-T 20.0     1.0     50.11 NaN   34.03 NaN              42.04   
H-UNet-HPCA        1.0      1.0     33.72 NaN   20.47 NaN                NaN   
H-UNet-HPCA-T      1.0      1.0     34.84 NaN   21.30 NaN                NaN   

                                       Average Surface Distance      
                                   std                     mean std  
Model              Inv Temp Regime                                   
H-UNet-BASE-SWTA   20.0     1.0    NaN                     7.20 NaN  
H-UNet-BASE-SWTA-T 20.0     1.0    NaN                     7.84 NaN  
H-UNet-HPCA        1.0      1.0    NaN                      NaN NaN  
H-UNet-HPCA-T      1.0      1.0    NaN                      NaN NaN

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [18]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2']       # regimes to be considered
IMAGE_SIZE = 480
INV_TEMP_GlaS=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=glas').glob('unet_base')),
    #'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=glas/unet/inv_temp-1').glob('regime-*')),
    ##'H-UNet-HPCA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft_fromBestDice/inv_temp-1').glob('regime-*')),
    ##'H-UNet-HPCA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft_fromBestDice/inv_temp-1').glob('regime-*')),
    #'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_ft/inv_temp-1').glob('regime-*')),
    #'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-hpca_t_ft/inv_temp-1').glob('regime-*')),
    #'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    #'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    ##'H-UNet-SWTA-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft_fromBestDice/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    ##'H-UNet-SWTA-T-FT-FROM-BEST-DICE': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft_fromBestDice/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet_base-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=glas/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_GlaS)).glob('regime-*')),
}

In [19]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)

In [20]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,UNet,0,1.0,0.2,82.31,70.79,21.56,2.75
1,UNet,1,1.0,0.2,79.69,67.33,22.69,3.06
2,UNet,2,1.0,0.2,77.99,65.01,25.59,3.20
3,UNet,3,1.0,0.2,80.72,68.60,25.77,3.25
4,UNet,4,1.0,0.2,82.35,70.79,22.47,2.91
5,UNet,5,1.0,0.2,75.64,61.99,37.45,5.45
6,UNet,6,1.0,0.2,78.85,66.22,25.13,3.66
7,UNet,7,1.0,0.2,80.72,68.97,30.89,4.10
8,UNet,8,1.0,0.2,79.55,67.09,22.18,3.25
9,UNet,9,1.0,0.2,78.94,66.35,27.56,3.78


Dice           Jaccard           Hausdorff Distance  \
                         mean       std    mean       std               mean   
Model Inv Temp Regime                                                          
UNet  1.0      0.2     79.676  2.017701  67.314  2.674805             26.129   

                                Average Surface Distance            
                            std                     mean       std  
Model Inv Temp Regime                                               
UNet  1.0      0.2     4.898783                    3.541  0.786617

<h1>PH2 Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [10]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_t')),
}

In [11]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv', regimes=['1.0']) for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

More than 1 tensorboard folder in run: /home/luca/datino/results/hebbian-medical-image-segmentation/runs/experiment=ph2/hunet_base-swta/inv_temp-1/regime-1.0/run-0
Skipping not found: /home/luca/datino/results/hebbian-medical-image-segmentation/runs/experiment=ph2/hunet_base-swta/inv_temp-1/regime-1.0/run-0/test_predictions/preds_from_last.csv
More than 1 tensorboard folder in run: /home/luca/datino/results/hebbian-medical-image-segmentation/runs/experiment=ph2/hunet_base-swta/inv_temp-10/regime-1.0/run-0
Skipping not found: /home/luca/datino/results/hebbian-medical-image-segmentation/runs/experiment=ph2/hunet_base-swta/inv_temp-10/regime-1.0/run-0/test_predictions/preds_from_last.csv
More than 1 tensorboard folder in run: /home/luca/datino/results/hebbian-medical-image-segmentation/runs/experiment=ph2/hunet_base-swta/inv_temp-100/regime-1.0/run-0
Skipping not found: /home/luca/datino/results/hebbian-medical-image-segmentation/runs/experiment=ph2/hunet_base-swta/inv_temp-100/regime-1.0

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

In [4]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_ft')),
    #'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet_base-swta_t_ft')),
    #'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/').glob('hunet-swta_t_ft')),
}

In [5]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [6]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-BASE-SWTA-FT,0,1.0,0.01,75.57,63.16,19.17,2.93
1,H-UNet-BASE-SWTA-FT,0,1.0,0.02,75.43,63.36,14.70,2.01
2,H-UNet-BASE-SWTA-FT,0,1.0,0.05,79.87,67.81,9.70,1.26
3,H-UNet-BASE-SWTA-FT,0,1.0,0.10,84.61,74.48,6.90,0.85
4,H-UNet-BASE-SWTA-FT,0,1.0,0.20,88.37,79.72,2.17,0.36
...,...,...,...,...,...,...,...,...
245,H-UNet-BASE-SWTA-FT,9,50.0,0.10,88.49,79.95,3.02,0.40
246,H-UNet-BASE-SWTA-FT,9,50.0,0.20,89.51,81.89,4.06,0.60
247,H-UNet-BASE-SWTA-FT,9,100.0,0.05,84.15,73.80,4.91,0.67
248,H-UNet-BASE-SWTA-FT,9,100.0,0.10,86.82,77.64,4.63,0.63


Dice           Jaccard            \
                                       mean       std    mean       std   
Model               Inv Temp Regime                                       
H-UNet-BASE-SWTA-FT 1.0      0.01    74.179  4.972635  62.108  5.522320   
                             0.02    78.688  5.130880  67.328  6.122891   
                             0.05    82.505  3.613014  71.528  4.804789   
                             0.10    86.587  2.337882  77.099  3.438664   
                             0.20    88.620  1.393756  80.036  2.137138   
                    5.0      0.01    73.640  5.600869  61.670  6.469745   
                             0.02    76.497  3.134837  64.407  4.085177   
                             0.05    79.990  3.040618  68.460  3.964770   
                             0.10    85.889  1.704200  76.304  2.274434   
                             0.20    88.838  1.005936  80.421  1.433321   
                    10.0     0.01    74.077  6.153487  62.015  7.069886   
                             0.02    76.633  5.896920  64.850  7.211314   
                             0.05    82.884  3.167678  72.309  4.268209   
                             0.10    86.804  1.941656  77.380  2.836230   
                             0.20    88.555  1.165230  80.125  1.653456   
                    20.0     0.01    73.571  4.654165  61.886  4.963668   
                             0.05    81.808  3.415767  70.811  4.592009   
                             0.10    87.516  1.371384  78.534  2.006623   
                             0.20    88.367  1.538203  79.862  2.105183   
                    50.0     0.05    81.926  3.714360  71.089  4.900085   
                             0.10    85.936  1.781274  76.387  2.417400   
                             0.20    89.555  0.737387  81.701  1.059533   
                    100.0    0.05    80.694  3.035311  69.584  3.942879   
                             0.10    86.719  1.304458  77.471  1.937429   
                             0.20    89.288  1.357692  81.256  2.040606   

                                    Hausdorff Distance            \
                                                  mean       std   
Model               Inv Temp Regime                                
H-UNet-BASE-SWTA-FT 1.0      0.01               17.414  5.258574   
                             0.02               11.925  7.275678   
                             0.05                6.579  3.649409   
                             0.10                3.844  1.647673   
                             0.20                2.498  0.487141   
                    5.0      0.01               16.812  8.138955   
                             0.02               13.386  4.802745   
                             0.05                8.204  4.112034   
                             0.10                5.220  2.697097   
                             0.20                2.550  0.890668   
                    10.0     0.01               18.285  6.933391   
                             0.02               14.529  9.802669   
                             0.05                5.877  2.924164   
                             0.10                3.720  1.098484   
                             0.20                3.047  1.331674   
                    20.0     0.01               17.211  5.594131   
                             0.05                7.400  3.405081   
                             0.10                3.981  1.362607   
                             0.20                3.684  2.236491   
                    50.0     0.05                7.854  3.641789   
                             0.10                4.522  1.815151   
                             0.20                3.100  1.145775   
                    100.0    0.05                8.062  2.988804   
                             0.10                4.393  1.878013   
                             0.20                2.967  1.484311   

                                 

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [39]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

INV_TEMP_PH2=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_PH2)).glob('run-*')),
}

In [40]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

Skipping not found: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=ph2/hunet_base-swta/inv_temp-20/regime-1.0/run-0/test_predictions/preds_from_last.csv
Skipping not found: /home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs/experiment=ph2/hunet_base-swta_t/inv_temp-20/regime-1.0/run-0/test_predictions/preds_from_last.csv


In [41]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

/tmp/ipykernel_1922413/265248669.py:24: RuntimeWarning: Mean of empty slice
  mean_value = np.nanmean(values)


,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-HPCA,0,1.0,1.0,21.89,12.59,104.8,41.46
1,H-UNet-HPCA-T,0,1.0,1.0,21.21,12.13,NaN,NaN


Dice     Jaccard     Hausdorff Distance      \
                                mean std    mean std               mean std   
Model         Inv Temp Regime                                                 
H-UNet-HPCA   1.0      1.0     21.89 NaN   12.59 NaN              104.8 NaN   
H-UNet-HPCA-T 1.0      1.0     21.21 NaN   12.13 NaN                NaN NaN   

                              Average Surface Distance      
                                                  mean std  
Model         Inv Temp Regime                               
H-UNet-HPCA   1.0      1.0                       41.46 NaN  
H-UNet-HPCA-T 1.0      1.0                         NaN NaN

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [13]:
EXP_ROOT = "/home/luca/datino/results/hebbian-medical-image-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2', '0.25']       # regimes to be considered
IMAGE_SIZE = 480
INV_TEMP_PH2=10          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=ph2').glob('unet_base')),
    ##'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=ph2').glob('unet')),
    ##'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2').glob('hunet_base-hpca_ft')),
    ##'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2').glob('hunet_base-hpca_t_ft')),
    ##'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=ph2').glob('hunet-hpca_ft')),
    ##'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2').glob('hunet-hpca_t_ft')),
    #'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    #'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet_base-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    #'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=ph2/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
}

In [12]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv', regimes=REGIMES, image_size=IMAGE_SIZE) for k, v in runs.items() for r in v], ignore_index=True)

Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Different image size in conf. Image size in conf: 480, Image size: 256
Differ

In [12]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard,Hausdorff Distance,Average Surface Distance
0,H-UNet-Pseudolabeling-HPCA-T-FT,0,1.0,0.01,58.14,46.18,109.55,20.28
1,H-UNet-Pseudolabeling-HPCA-T-FT,0,1.0,0.02,69.58,58.64,34.29,6.45
2,H-UNet-Pseudolabeling-HPCA-T-FT,0,1.0,0.05,89.10,81.19,4.51,0.66
3,H-UNet-Pseudolabeling-HPCA-T-FT,0,1.0,0.10,86.89,79.07,9.14,1.67
4,H-UNet-Pseudolabeling-HPCA-T-FT,0,1.0,0.20,89.18,81.63,7.69,1.30
5,H-UNet-Pseudolabeling-HPCA-T-FT,0,1.0,0.25,90.60,83.59,6.20,0.80
6,H-UNet-Pseudolabeling-HPCA-T-FT,1,1.0,0.01,69.20,55.30,68.55,11.39
7,H-UNet-Pseudolabeling-HPCA-T-FT,1,1.0,0.02,84.72,75.51,6.77,0.98
8,H-UNet-Pseudolabeling-HPCA-T-FT,1,1.0,0.05,88.87,81.56,3.94,0.63
9,H-UNet-Pseudolabeling-HPCA-T-FT,1,1.0,0.10,89.74,82.17,7.17,0.80


Dice            Jaccard  \
                                                   mean        std    mean   
Model                           Inv Temp Regime                              
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.01    61.618  13.307067  49.796   
                                         0.02    79.876   6.880196  69.569   
                                         0.05    86.555   2.014190  77.999   
                                         0.10    89.122   1.649605  81.765   
                                         0.20    91.413   0.949082  84.589   
                                         0.25    91.933   0.851026  85.510   

                                                           Hausdorff Distance  \
                                                       std               mean   
Model                           Inv Temp Regime                                 
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.01    13.774789             71.141   
                                         0.02     7.920030             15.256   
                                         0.05     2.528190              7.715   
                                         0.10     1.965702              5.822   
                                         0.20     1.361979              3.649   
                                         0.25     1.239077              3.385   

                                                            \
                                                       std   
Model                           Inv Temp Regime              
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.01    34.119033   
                                         0.02     9.087026   
                                         0.05     3.298910   
                                         0.10     2.557706   
                                         0.20     1.953393   
                                         0.25     1.542662   

                                                Average Surface Distance  \
                                                                    mean   
Model                           Inv Temp Regime                            
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.01                     13.184   
                                         0.02                      2.534   
                                         0.05                      1.158   
                                         0.10                      0.930   
                                         0.20                      0.508   
                                         0.25                      0.469   

                                                           
                                                      std  
Model                           Inv Temp Regime            
H-UNet-Pseudolabeling-HPCA-T-FT 1.0      0.01    6.553010  
                                         0.02    1.895680  
                                         0.05    0.577770  
                                         0.10    0.474997  
                                         0.20    0.315834  
                                         0.25    0.195644

<h1>KvasirSEG Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [ ]:
# Unsupervised learning models are evaluate over the whole dataset while fine tuned models over a fraction of data

EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet_base-swta')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet_base-swta_t')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_all(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

In [ ]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

runs = {
    # UNET-based
    'H-UNet-BASE-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet_base-swta_ft')),
    'H-UNet-BASE-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet_base-swta_t_ft')),
    'H-UNet-Pseudolabeling-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet-swta_t')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/').glob('hunet-swta_t_ft')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'preds.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

<h2>Evaluation - Hebbian Unsupervised Pretraining</h2>

<p>Evaluate Hebbian models pretrained in an unsupervised way over the datasets; only best temperature values for SWTA are considered.</p>

In [ ]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

INV_TEMP_KVASIR=20          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'H-UNet-HPCA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-hpca/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-HPCA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-hpca_t/inv_temp-1/regime-1.0').glob('run-*')),
    'H-UNet-BASE-SWTA': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-swta/inv_temp-{}/regime-1.0'.format(INV_TEMP_KVASIR)).glob('run-*')),
    'H-UNet-BASE-SWTA-T': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-swta_t/inv_temp-{}/regime-1.0'.format(INV_TEMP_KVASIR)).glob('run-*')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset; only best temperature values for SWTA are considered.</p>

In [ ]:
EXP_ROOT = "/home/luca/dgx-a100/raid/home/lucaciampi/workspace/hebbian-skin-cancer-segmentation/runs"

REGIMES = ['0.01', '0.02', '0.05', '0.1', '0.2', '0.25']       # regimes to be considered
INV_TEMP_KVASIR=10          # to be set accordingly, used by SWTA

runs = {
    # UNET-based
    'UNet': list(Path(EXP_ROOT + '/experiment=kvasirSEG/unet_base/inv_temp-1').glob('regime-*')),
    'UNet-Pseudolabeling': list(Path(EXP_ROOT + '/experiment=kvasirSEG/unet/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-Pseudolabeling-HPCA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-hpca_t_ft/inv_temp-1').glob('regime-*')),
    'H-UNet-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet_base-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
    'H-UNet-Pseudolabeling-SWTA-T-FT': list(Path(EXP_ROOT + '/experiment=kvasirSEG/hunet-swta_t_ft/inv_temp-{}'.format(INV_TEMP_PH2)).glob('regime-*')),
}

In [ ]:
# Collect predictions scanning runs
predictions = pd.concat([collect_one(k, r, 'preds_from_last.csv') for k, v in runs.items() for r in v], ignore_index=True)
#predictions = pd.concat([collect_one(k, r, 'preds_from_best_dice.csv') for k, v in runs.items() for r in v], ignore_index=True)

In [ ]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper)

display(metrics)

summary = summarize_metrics(metrics)

display(summary)